In [1]:
!pip install -U accelerate
!pip install -U transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install rouge_score
!pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import pandas as pd
from tqdm import tqdm
from PIL import Image
from transformers import AutoTokenizer, AutoImageProcessor
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.cuda as cuda

# Check if GPU is available
if cuda.is_available():
    device = torch.device('cuda')
    print(f'Using GPU: {device}')
else:
    device = torch.device('cpu')
    print('Using CPU')

text_data = pd.read_csv("D:\MS COCO\captions.txt", sep='|')
text_data.rename(columns={"comment_number": "image_repeat", "comment": "caption"}, inplace=True)

text_data_train = text_data.drop(text_data[text_data['image_repeat'] == 0].index)
text_data_train.reset_index(drop=True, inplace=True)

text_data_val = text_data[text_data['image_repeat'] == 0]
text_data_val.reset_index(drop=True, inplace=True)

# Create directories to store the preprocessed images and captions
image_dir = 'D:\MS COCO\preprocessed_images'
caption_dir = 'D:\MS COCO\preprocessed_captions'
os.makedirs(image_dir, exist_ok=True)
os.makedirs(caption_dir, exist_ok=True)

# Preprocess the images and captions
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
decoder_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", add_special_tokens=True)

if decoder_tokenizer.pad_token is None:
    decoder_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

missing_images = []
missing_captions = []

progress_bar = tqdm(range(len(text_data_train) // 4), desc='Preprocessing')
for index in range(0, len(text_data_train), 4):
    path_filename = "D:/MS COCO/Images/" + text_data_train.iloc[index]['image_name']
    if os.path.exists(path_filename):
        image = Image.open(path_filename).convert('RGB')
        image_tensor = image_processor(images=[image], return_tensors='pt').pixel_values.to(device)
        torch.save(image_tensor, os.path.join(image_dir, f'{text_data_train.iloc[index]["image_name"].split(".")[0]}.pt'))
    else:
        missing_images.append(text_data_train.iloc[index]['image_name'])

    for i in range(4):
        if index + i + 1 < len(text_data_train):
            caption = str(text_data_train.iloc[index + i + 1]['caption'])
            caption_token = decoder_tokenizer(caption, add_special_tokens=True, padding="max_length", max_length=32, truncation=True).input_ids
            caption_token = [token if token != decoder_tokenizer.pad_token_id else -100 for token in caption_token]
            caption_file = os.path.join(caption_dir, f'{text_data_train.iloc[index]["image_name"].split(".")[0]}_{i}.txt')
            try:
                with open(caption_file, 'w') as f:
                    f.write(' '.join(map(str, caption_token)))
            except FileNotFoundError:
                missing_captions.append(caption_file)
    progress_bar.update(1)

if missing_images:
    print(f"Missing images: {', '.join(missing_images)}")
if missing_captions:
    print(f"Missing captions: {', '.join(missing_captions)}")

<>:19: SyntaxWarning: invalid escape sequence '\M'
<>:29: SyntaxWarning: invalid escape sequence '\M'
<>:30: SyntaxWarning: invalid escape sequence '\M'
<>:19: SyntaxWarning: invalid escape sequence '\M'
<>:29: SyntaxWarning: invalid escape sequence '\M'
<>:30: SyntaxWarning: invalid escape sequence '\M'
C:\Users\jeeva\AppData\Local\Temp\ipykernel_25172\3235328685.py:19: SyntaxWarning: invalid escape sequence '\M'
  text_data = pd.read_csv("D:\MS COCO\captions.txt", sep='|')
C:\Users\jeeva\AppData\Local\Temp\ipykernel_25172\3235328685.py:29: SyntaxWarning: invalid escape sequence '\M'
  image_dir = 'D:\MS COCO\preprocessed_images'
C:\Users\jeeva\AppData\Local\Temp\ipykernel_25172\3235328685.py:30: SyntaxWarning: invalid escape sequence '\M'
  caption_dir = 'D:\MS COCO\preprocessed_captions'
C:\Users\jeeva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please u

Using GPU: cuda


Preprocessing: 100%|█████████▉| 155064/155069 [56:28<00:00, 40.17it/s]  

Preprocessing: 100%|██████████| 155069/155069 [56:43<00:00, 40.17it/s]